<a href="https://colab.research.google.com/github/ttrevorreese/finalyearproject/blob/main/Final%20Map%20Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Implementation

---

## Import Libraries

In [15]:
from array import array
import pandas as pd
import time
import math

## Loading Dataset

In [16]:
# Import data from csv
data = pd.read_csv('/content/drive/MyDrive/! University of Roehampton/Year 3/! Final Year Project/Datasets/BusStops_v1 - Sheet1.csv')

# Drop unnecessary columns (can be utilized in further development)
data.drop(['Scores', 'Notes'], axis=1, inplace=True)

# Display a preview of the first 10 rows of the data
data.head(10)

,Stop Name,Latitude,Longitude,Classification
0,Heritage Marketplace,33.640179,-117.168344,Commercial
1,Sonoma at Mapleton Apartments,33.631778,-117.169029,Residential
2,Mapleton & Iris,33.632067,-117.160945,Residential
3,Whitewood & Mapleton,33.632119,-117.154617,Residential
4,Poinsettia & Verbena,33.629445,-117.151333,Residential
5,Mapleton Park,33.628591,-117.163263,Community
6,Kaiser Permanente @ Keller,33.627069,-117.168017,Services
7,Springbrook Park,33.626276,-117.162660,Community
8,Keller & Whitewood,33.626974,-117.157986,Residential
9,Meadowlark & Brians,33.623213,-117.162675,Residential


In [17]:
# Storing latitude and longitude into a list each
lat = data["Latitude"]
lon = data["Longitude"]
name = data["Stop Name"]
clsf = data["Classification"]

## Adjacency List

In [18]:
import math

# This graph contains all of the bus stops and their connecting nodes with each of the distances between them measured in miles, collected from Google Maps.
graph = {
    "Heritage Marketplace": {"Sonoma at Mapleton Apartments": 0.7, "Antelope & Keller": 0.9},
    "Sonoma at Mapleton Apartments": {"Heritage Marketplace": 0.7, "Antelope & Keller": 0.4, "Mapleton & Iris": 0.5, "Mapleton Park": 0.5},
    "Mapleton & Iris": {"Sonoma at Mapleton Apartments": 0.5, "Whitewood & Mapleton": 0.4, "Mapleton Park": 0.5},
    "Whitewood & Mapleton": {"Poinsettia & Verbena": 0.3, "Mapleton & Iris": 0.4, "Keller & Whitewood": 0.4},
    "Poinsettia & Verbena": {"Whitewood & Mapleton": 0.3, "Keller & Whitewood": 0.6},
    "Mapleton Park": {"Sonoma at Mapleton Apartments": 0.5, "Mapleton & Iris": 0.5},
    "Kaiser Permanente @ Keller": {"Antelope & Keller": 0.1, "Springbrook Park": 0.4, "Keller & Whitewood": 0.6},
    "Springbrook Park": {"Kaiser Permanente @ Keller": 0.4, "Keller & Whitewood": 0.3, "Meadowlark & Brians": 0.2},
    "Keller & Whitewood": {"Poinsettia & Verbena": 0.6, "Whitewood & Mapleton": 0.4, "Kaiser Permanente @ Keller": 0.6, "Springbrook Park": 0.3, "Alderwood Park": 1.1, "Meadowlark & Brians": 0.6},
    "Meadowlark & Brians": {"Keller & Whitewood": 0.6, "Springbrook Park": 0.2},
    "Antelope & Keller": {"Sonoma at Mapleton Apartments": 0.4, "Kaiser Permanente @ Keller": 0.1, "Loma Linda University Medical Center": 0.9, "Heritage Marketplace": 0.9},
    "Loma Linda University Medical Center": {"Antelope & Keller": 0.9, "Alderwood Park": 0.5},
    "Alderwood Park": {"Keller & Whitewood": 1.1, "Loma Linda University Medical Center": 0.5, "Whitewood & Lee/Linnel": 0.5},
    "Whitewood & Lee/Linnel": {"Alderwood Park": 0.5, "Meadowlark Pointe": 0.2, "McElwain @ Super Target": 0.9},
    "Meadowlark Pointe": {"Whitewood & Lee/Linnel": 0.2, "Clinton Keith & Whitewood": 0.3},
    "McElwain @ Super Target": {"Whitewood & Lee/Linnel": 0.9, "Orchard Center": 0.2},
    "Orchard Center": {"McElwain @ Super Target": 0.2, "Clinton Keith @ Costco & Vista Murrieta High School": 0.7, "Clinton Keith & Murrieta Oaks/Greer": 0.7, "Antelope Hills Elementary School": 0.5},
    "Clinton Keith & Whitewood": {"Meadowlark Pointe": 0.3, "Clinton Keith @ Costco & Vista Murrieta High School": 0.4, "Clinton Keith & Menifee": 0.5, "Whitewood & Hunter": 0.9},
    "Clinton Keith @ Costco & Vista Murrieta High School": {"Clinton Keith & Whitewood": 0.4, "Orchard Center": 0.7},
    "Clinton Keith & Menifee": {"Clinton Keith & Whitewood": 0.5, "Clinton Keith & Trois Valley": 1.2, "Los Alamos & Shannon N": 0.7},
    "Los Alamos & Shannon N": {"Clinton Keith & Menifee": 0.7, "Los Alamos & Mason": 0.4},
    "Los Alamos & Mason": {"Los Alamos & Shannon N": 0.4, "Rail Ranch Elementary School": 0.7, "Los Alamos Hills Sports Park": 0.9},
    "Rail Ranch Elementary School": {"Los Alamos & Mason": 0.7, "Los Alamos Hills Sports Park": 0.4, "Murrieta Gateway": 0.4, "Whitewood & Busman": 0.8, "Whitewood & Willowbend": 0.6},
    "Los Alamos Hills Sports Park": {"Rail Ranch Elementary School": 0.4, "Whitewood & Busman": 0.5, "Whitewood & Hunter": 0.4, "Los Alamos & Mason": 0.9} ,
    "Whitewood & Hunter": {"Clinton Keith & Whitewood": 0.9, "Los Alamos Hills Sports Park": 0.4, "Whitewood & Busman": 0.6},
    "Clinton Keith & Murrieta Oaks/Greer": {"Orchard Center": 0.7, "Antelope Hills Elementary School": 0.5, "Murrieta Antelope Hills Park": 0.3, "Greer & Pabesu East": 0.6, "Clinton Keith & California Oaks": 0.6},
    "Murrieta Antelope Hills Park": {"Antelope Hills Elementary School": 0.4, "Clinton Keith & Murrieta Oaks/Greer": 0.3, "Murrieta Oaks & Oak Meadows": 0.4},
    "Murrieta Oaks & Oak Meadows": {"Murrieta Antelope Hills Park": 0.4, "Antelope Hills Elementary School": 0.5},
    "Antelope Hills Elementary School": {"Orchard Center": 0.5, "Murrieta Oaks & Oak Meadows": 0.5, "Clinton Keith & Murrieta Oaks/Greer": 0.5, "Murrieta Antelope Hills Park": 0.4},
    "Greer & Pabesu East": {"Clinton Keith & Murrieta Oaks/Greer": 0.6, "Greer & Pabesu West": 0.7},
    "Greer & Pabesu West": {"Greer & Pabesu East": 0.7, "Blackmore Ranch Park": 0.5},
    "Clinton Keith & California Oaks": {"Clinton Keith & Murrieta Oaks/Greer": 0.6, "California Oaks & Falconer/Manresa": 0.4, "Nutmeg & St Raphael Tovashal": 0.5, "Blackmore Ranch Park": 0.6, "Clinton Keith & Spinning Wheel": 0.9},
    "Blackmore Ranch Park": {"Greer & Pabesu West": 0.5, "Clinton Keith & California Oaks": 0.6, "Clinton Keith & Spinning Wheel": 0.7, "Nutmeg & St Raphael Tovashal": 0.3},
    "Clinton Keith & Spinning Wheel": {"Clinton Keith & California Oaks": 0.9, "Nutmeg & St Raphael Tovashal": 0.6, "Clinton Keith & Smith Ranch": 0.7, "Blackmore Ranch Park": 0.7},
    "Nutmeg & St Raphael Tovashal": {"Blackmore Ranch Park": 0.3, "Clinton Keith & California Oaks": 0.5, "Clinton Keith & Spinning Wheel": 0.6, "Nutmeg & Gingerbread": 0.4},
    "Nutmeg & Gingerbread": {"Nutmeg & St Raphael Tovashal": 0.4, "Golden Pheasant and Malacca/Tanager": 0.4, "Nutmeg & Lincoln": 0.4},
    "Golden Pheasant and Malacca/Tanager": {"Nutmeg & Gingerbread": 0.4, "Valley Vista Park": 0.5},
    "Valley Vista Park": {"Golden Pheasant and Malacca/Tanager": 0.5, "California Oaks & Falconer/Manresa": 0.4},
    "Whitewood & Busman": {"Murrieta Gateway": 0.7, "Whitewood & Hunter": 0.6, "Los Alamos Hills Sports Park": 0.5, "Rail Ranch Elementary School": 0.8},
    "Murrieta Gateway": {"Whitewood & Busman": 0.7, "Rail Ranch Elementary School": 0.4, "Whitewood & Willowbend": 0.4, "Sierra Vista Shopping Center": 0.4},
    "Whitewood & Willowbend": {"Murrieta Gateway": 0.4, "Rail Ranch Elementary School": 0.6, "Whitewood & Blackthorne": 0.3, "Rancho Acacias Park @ Avenida Palizada": 0.4},
    "Rancho Acacias Park @ Avenida Palizada": {"Whitewood & Willowbend": 0.4, "Whitewood & Blackthorne": 0.5},
    "Whitewood & Blackthorne": {"Whitewood & Willowbend": 0.3, "Rancho Acacias Park @ Avenida Palizada": 0.5, "Whitewood Park": 0.3},
    "Whitewood Park": {"Whitewood & Blackthorne": 0.3, "Alta Murrieta Elementary School": 0.2},
    "Alta Murrieta Elementary School": {"Whitewood Park": 0.2, "Whitewood & Via Temprano/Highbury": 0.3, "Alta Murrieta & Ridgeplume": 0.4},
    "Whitewood & Via Temprano/Highbury": {"Alta Murrieta Elementary School": 0.3, "Rockcrest & Daphne": 0.5, "Firefighters Park": 0.7},
    "Firefighters Park": {"Whitewood & Via Temprano/Highbury": 0.7, "Murrieta Hot Springs & Alta Murrieta Sam's Club": 0.5, "Murrieta Hot Springs & Margarita": 0.7, "Jackson & Magellan": 0.7},
    "Murrieta Hot Springs & Alta Murrieta Sam's Club": {"Firefighters Park": 0.5, "Alta Murrieta Sports Park": 0.4, "Rancho Springs Medical Center": 0.5, "Jackson & Magellan": 0.7, "Murrieta Springs Plaza": 1.4},
    "Alta Murrieta Sports Park": {"Murrieta Hot Springs & Alta Murrieta Sam's Club": 0.4, "Alta Murrieta & Ridgeplume": 0.2, "Rockcrest & Daphne": 0.5},
    "Alta Murrieta & Ridgeplume": {"Alta Murrieta Sports Park": 0.2, "Alta Murrieta Elementary School": 0.4},
    "Rockcrest & Daphne": {"Whitewood & Via Temprano/Highbury": 0.5, "Alta Murrieta Sports Park": 0.5},
    "Jackson & Magellan": {"Murrieta Hot Springs & Alta Murrieta Sam's Club": 0.7, "Firefighters Park": 0.7, "Murrieta Palms @ Ynez": 0.8},
    "Murrieta Palms @ Ynez": {"Jackson & Magellan": 0.8},
    "Murrieta Hot Springs & Margarita": {"Firefighters Park": 0.7, "Murrieta Hot Springs & Calle Del Lago": 0.6, "Margarita & Torrey Pines": 0.5},
    "Murrieta Hot Springs & Calle Del Lago": {"Murrieta Hot Springs & Margarita": 0.6, "Murrieta Hot Springs & Winchester Hwy 79": 0.4},
    "Murrieta Hot Springs & Winchester Hwy 79": {"Murrieta Hot Springs & Calle Del Lago": 0.4, "Winchester & Robert Trent Jones/Technology": 0.6},
    "Winchester & Robert Trent Jones/Technology": {"Murrieta Hot Springs & Winchester Hwy 79": 0.6, "Winchester & Hunter": 0.6},
    "Winchester & Hunter": {"Winchester & Robert Trent Jones/Technology": 0.6, "Hunter & Via Mira Mosa": 0.7, "Winchester & La Alba/Sparkman": 0.5},
    "Winchester & La Alba/Sparkman": {"Winchester & Hunter": 0.5, "French Valley Airport @ Winchester": 0.4, "Palomar Park & La Alba Trail": 0.4},
    "French Valley Airport @ Winchester": {"Winchester & La Alba/Sparkman": 0.4, "French Valley Village Shopping Center": 0.8, "Palomar Park & La Alba Trail": 0.8},
    "French Valley Village Shopping Center": {"French Valley Airport @ Winchester": 0.8, "Winchester & Max Gilliss/Thompson": 0.6},
    "Winchester & Max Gilliss/Thompson": {"Spencer's Crossing Sports Park": 0.5, "French Valley Village Shopping Center": 0.6},
    "Spencer's Crossing Sports Park": {"Winchester & Max Gilliss/Thompson": 0.5, "Clinton Keith & Trois Valley": 0.7},
    "Clinton Keith & Trois Valley": {"Clinton Keith & Menifee": 1.2, "Spencer's Crossing Sports Park": 0.7},
    "Clinton Keith & Smith Ranch": {"Clinton Keith & Spinning Wheel": 0.7, "Clinton Keith Veterinary Hospital": 0.5},
    "Clinton Keith Veterinary Hospital": {"Clinton Keith & Smith Ranch": 0.5, "Clinton Keith & Wildomar Trail": 0.5, "Inland Valley Medical Center": 0.6},
    "Inland Valley Medical Center": {"Clinton Keith Veterinary Hospital": 0.6, "Clinton Keith & Wildomar Trail": 0.6, "Prielipp & Elizabeth": 0.6},
    "Clinton Keith & Wildomar Trail": {"Clinton Keith Veterinary Hospital": 0.5, "Inland Valley Medical Center": 0.6, "Clinton Keith & Hidden Springs": 0.4},
    "Prielipp & Elizabeth": {"Inland Valley Medical Center": 0.6, "Jackson & Robards/Eden Springs": 0.4},
    "Nutmeg & Lincoln": {"Nutmeg & Gingerbread": 0.4, "Glen Arbor Park": 0.2},
    "Glen Arbor Park": {"Nutmeg & Lincoln": 0.2, "Jackson & Robards/Eden Springs": 0.3, "Nutmeg & Morning Glory": 0.2, "Jackson & Via Diamante/Buttercup": 0.3},
    "Jackson & Robards/Eden Springs": {"Glen Arbor Park": 0.3, "Prielipp & Elizabeth": 0.4},
    "Clinton Keith & Hidden Springs": {"Clinton Keith & Wildomar Trail": 0.4, "Renaissance Plaza": 0.3},
    "Renaissance Plaza": {"Clinton Keith & Hidden Springs": 0.3, "Clinton Keith & Palomar": 0.1},
    "Clinton Keith & Palomar": {"Renaissance Plaza": 0.1, "Sycamore Academy Charter School": 0.4, "Clinton Keith & Grand": 0.5},
    "Sycamore Academy Charter School": {"Clinton Keith & Palomar": 0.4, "Washington & Grizzly Ridge": 0.8},
    "Clinton Keith & Grand": {"Clinton Keith & Palomar": 0.5, "Clinton Keith & Club View": 0.4},
    "Clinton Keith & Calle Del Oso Oro Bear Creek": {"Clinton Keith & Club View": 0.3, "Riverside County Fire Department Station 75": 0.8, "Club View & Via Moreno": 0.5, "Calle Del Oso Oro & Via Moreno": 0.8},
    "Riverside County Fire Department Station 75": {"Clinton Keith & Calle Del Oso Oro Bear Creek": 0.8},
    "Clinton Keith & Club View": {"Club View & Via Moreno": 0.4, "Clinton Keith & Calle Del Oso Oro Bear Creek": 0.3, "Clinton Keith & Grand": 0.4},
    "Club View & Via Moreno": {"Clinton Keith & Calle Del Oso Oro Bear Creek": 0.5, "Clinton Keith & Club View": 0.4, "Cole Canyon Elementary School": 0.3},
    "Cole Canyon Elementary School": {"Club View & Via Moreno": 0.3, "Rosewood Park": 0.4, "Calle Del Oso Oro & Via Moreno": 0.4},
    "Calle Del Oso Oro & Via Moreno": {"Clinton Keith & Calle Del Oso Oro Bear Creek": 0.8, "Cole Canyon Elementary School": 0.4, "Calle Del Oso Oro & Oakleaf/Vineyard": 0.3},
    "Rosewood Park": {"Cole Canyon Elementary School": 0.4, "Calle Del Oso Oro & Oakleaf/Vineyard": 0.4},
    "Calle Del Oso Oro & Oakleaf/Vineyard": {"Rosewood Park": 0.4, "Calle Del Oso Oro & Via Moreno": 0.3, "Calle Del Oso Oro & Applewood": 0.4},
    "Calle Del Oso Oro & Applewood": {"Calle Del Oso Oro & Oakleaf/Vineyard": 0.4, "Washington & Calle Del Oso Oro/Nutmeg": 0.3},
    "Washington & Calle Del Oso Oro/Nutmeg": {"Calle Del Oso Oro & Applewood": 0.3, "Washington & Grizzly Ridge": 0.4, "Nutmeg & Jefferson": 0.6, "Plaza de Oro": 0.1},
    "Washington & Grizzly Ridge": {"Washington & Calle Del Oso Oro/Nutmeg": 0.4, "Grizzly Ridge & Camellia": 0.3, "Sycamore Academy Charter School": 0.8},
    "Nutmeg & Jefferson": {"Gateway Church of the Nazarene": 0.6, "Washington & Calle Del Oso Oro/Nutmeg": 0.6, "Nutmeg & Via Olivia": 0.4, "Jefferson & Magnolia/Nighthawk": 0.4},
    "Gateway Church of the Nazarene": {"Nutmeg & Jefferson": 0.6, "Grizzly Ridge & Camellia": 0.3},
    "Grizzly Ridge & Camellia": {"Gateway Church of the Nazarene": 0.3, "Washington & Grizzly Ridge": 0.3},
    "Nutmeg & Via Olivia": {"Nutmeg & Jefferson": 0.4, "Nutmeg & Morning Glory": 0.3},
    "Nutmeg & Morning Glory": {"Nutmeg & Via Olivia": 0.3, "Glen Arbor Park": 0.2},
    "Palomar Park & La Alba Trail": {"Monte Vista Elementary School": 0.4, "Winchester & La Alba/Sparkman": 0.4, "French Valley Airport @ Winchester": 0.8},
    "Monte Vista Elementary School": {"Palomar Park & La Alba Trail": 0.4, "Hunter & Via Mira Mosa": 0.7},
    "Hunter & Via Mira Mosa": {"Monte Vista Elementary School": 0.7, "Winchester & Hunter": 0.7},
    "Margarita & Torrey Pines": {"Murrieta Hot Springs & Margarita": 0.5, "Buchanan Elementary School": 0.3},
    "California Oaks & Falconer/Manresa": {"Clinton Keith & California Oaks": 0.4, "Valley Vista Park": 0.4, "California Oaks & Hancock": 0.6},
    "California Oaks & Hancock": {"California Oaks Sports Park": 0.5, "Hancock & Las Brisas N/Ridgewood": 0.3, "California Oaks & Falconer/Manresa": 0.6},
    "California Oaks Sports Park": {"California Oaks & Hancock": 0.5, "Shivela Middle School": 0.2, "California Oaks & Jackson": 0.3},
    "California Oaks & Jackson": {"California Oaks Sports Park": 0.3, "Jackson & Monroe": 0.6, "California Oaks Plaza": 0.3, "Monroe & Sunflower": 0.3},
    "California Oaks Plaza": {"California Oaks & Jackson": 0.3, "Calvary Chapel Murrieta": 0.6, "Kalmia @ Kohl's": 0.4, "Madison Center": 0.5, "Monroe & Sunflower": 0.2},
    "Calvary Chapel Murrieta": {"California Oaks Plaza": 0.6, "Chaco Canyon & Verdun": 0.5, "Monroe & Chatenay": 0.2, "Monroe & Sunflower": 0.6},
    "Monroe & Chatenay": {"Jackson & Monroe": 0.2, "Calvary Chapel Murrieta": 0.2, "Chaco Canyon & Verdun": 0.5},
    "Jackson & Monroe": {"Monroe & Chatenay": 0.2, "California Oaks & Jackson": 0.6, "E. Hale Curran Elementary School": 0.2},
    "E. Hale Curran Elementary School": {"Jackson & Monroe": 0.2, "Chaco Canyon & Verdun": 0.3, "Jackson & Via Diamante/Buttercup": 0.2},
    "Chaco Canyon & Verdun": {"E. Hale Curran Elementary School": 0.3, "Monroe & Chatenay": 0.5, "Calvary Chapel Murrieta": 0.5},
    "Jackson & Via Diamante/Buttercup": {"Glen Arbor Park": 0.3, "E. Hale Curran Elementary School": 0.2},
    "Kalmia @ Kohl's": {"Village Walk Plaza": 0.3, "Kalmia & Jefferson": 0.1, "California Oaks Plaza": 0.4, "Madison Center": 0.3},
    "Village Walk Plaza": {"Kalmia @ Kohl's": 0.3, "Jefferson & Lemon": 0.4, "Kalmia & Jefferson": 0.4},
    "Kalmia & Jefferson": {"Kalmia @ Kohl's": 0.1, "Village Walk Plaza": 0.4, "Jefferson & Lemon": 0.5, "Kalmia & Adams": 0.2, "Murrieta Town Square": 0.1},
    "Murrieta Town Square": {"Jefferson & Juniper": 0.1, "Kalmia & Jefferson": 0.1},
    "Jefferson & Lemon": {"Jefferson & Magnolia/Nighthawk": 0.5, "Lemon & Adams": 0.3, "Village Walk Plaza": 0.4, "Kalmia & Jefferson": 0.5},
    "Jefferson & Magnolia/Nighthawk": {"Nutmeg & Jefferson": 0.4, "Nighthawk & Adams": 0.2, "Jefferson & Lemon": 0.5},
    "Lemon & Adams": {"Jefferson & Lemon": 0.3, "Kalmia & Adams": 0.5, "Washington & Vineyard/Lemon": 0.3, "Nighthawk & Adams": 0.5},
    "Nighthawk & Adams": {"Washington & Nighthawk": 0.3, "Jefferson & Magnolia/Nighthawk": 0.2, "Lemon & Adams": 0.5},
    "Plaza de Oro": {"Washington & Nighthawk": 0.3, "Washington & Calle Del Oso Oro/Nutmeg": 0.1},
    "Washington & Nighthawk": {"Plaza de Oro": 0.3, "Nighthawk & Adams": 0.3, "Washington & Vineyard/Lemon": 0.5, "Murrieta Valley High School": 0.3},
    "Murrieta Valley High School": {"Washington & Nighthawk": 0.3, "Thompson Middle School & Murrieta Canyon Academy": 0.3},
    "Thompson Middle School & Murrieta Canyon Academy": {"Murrieta Valley High School": 0.3, "Vineyard & Luna Brilla/Calle Estancia": 0.5},
    "Vineyard & Luna Brilla/Calle Estancia": {"Thompson Middle School & Murrieta Canyon Academy": 0.5, "Washington & Vineyard/Lemon": 0.4},
    "Washington & Vineyard/Lemon": {"Vineyard & Luna Brilla/Calle Estancia": 0.4, "Centerpoint Church": 0.2, "Lemon & Adams": 0.3, "Washington & Nighthawk": 0.5},
    "Centerpoint Church": {"Washington & Vineyard/Lemon": 0.2, "Washington & Kalmia": 0.3},
    "Washington & Kalmia": {"Centerpoint Church": 0.3, "Kalmia & 2nd": 0.1, "Washington & B": 0.2, "Kalmia & Adams": 0.3},
    "Kalmia & Adams": {"Washington & Kalmia": 0.3, "Lemon & Adams": 0.5, "Murrieta Public Library & Murrieta Elementary School": 0.2, "Kalmia & Jefferson": 0.2},
    "Murrieta Public Library & Murrieta Elementary School": {"Kalmia & Adams": 0.2, "Murrieta Senior Center": 0.1, "Washington & B": 0.3},
    "Washington & B": {"Murrieta Public Library & Murrieta Elementary School": 0.3, "Washington & Juniper": 0.1, "Washington & Kalmia": 0.2, "B & 2nd": 0.1},
    "B & 2nd": {"Washington & B": 0.1, "Juniper & 2nd": 0.1, "Kalmia & 2nd": 0.2},
    "Kalmia & 2nd": {"Washington & Kalmia": 0.1, "Kalmia & 4th": 0.2, "B & 2nd": 0.2},
    "Murrieta Senior Center": {"Murrieta Public Library & Murrieta Elementary School": 0.1, "Jefferson & Juniper": 0.2, "Ivy & Adams": 0.3},
    "Washington & Juniper": {"Washington & B": 0.1, "Juniper & 2nd": 0.1, "Washington & Ivy": 0.3},
    "Juniper & 2nd": {"Washington & Juniper": 0.1, "B & 2nd": 0.1, "Washington & Ivy": 0.4},
    "Kalmia & 4th": {"Kalmia & 2nd": 0.2, "Kalmia & Hayes": 0.2},
    "Kalmia & Hayes": {"Kalmia & 4th": 0.2, "Hayes & Juniper": 0.3},
    "Hayes & Juniper": {"Kalmia & Hayes": 0.3, "Hayes & Ivy": 0.3},
    "Hayes & Ivy": {"Hayes & Juniper": 0.3, "Washington & Ivy": 0.5, "Hayes & Hawthorn": 0.5},
    "Hayes & Hawthorn": {"Hayes & Ivy": 0.5, "Hayes & De Luz/Guava": 0.5},
    "Hayes & De Luz/Guava": {"Hayes & Hawthorn": 0.5, "Adams & Guava": 0.8},
    "Washington & Ivy": {"Juniper & 2nd": 0.4, "Washington & Hawthorn": 0.5, "Hayes & Ivy": 0.5, "Washington & Juniper": 0.3},
    "Ivy & Adams": {"Washington & Ivy": 0.3, "Washington & Hawthorn": 0.8, "Jefferson & Ivy/Los Alamos": 0.2, "Murrieta Senior Center": 0.3},
    "Washington & Hawthorn": {"Washington & Ivy": 0.5, "Ivy & Adams": 0.8, "Adams & Guava": 0.7},
    "Adams & Guava": {"Washington & Hawthorn": 0.7, "Jefferson & Guava": 0.2, "Hayes & De Luz/Guava": 0.8},
    "Jefferson & Juniper": {"Murrieta Senior Center": 0.2, "Jefferson & Ivy/Los Alamos": 0.3, "Murrieta Town Square": 0.1, "Madison & Juniper": 0.2},
    "Jefferson & Ivy/Los Alamos": {"Jefferson & Juniper": 0.3, "Los Alamos & Madison": 0.3, "Ivy & Adams": 0.2, "Jefferson & Murrieta Hot Springs": 0.5},
    "Jefferson & Murrieta Hot Springs": {"Jefferson & Ivy/Los Alamos": 0.5, "Jefferson & Guava": 0.5, "Murrieta Springs Plaza": 0.4},
    "Jefferson & Guava": {"Jefferson & Murrieta Hot Springs": 0.5, "Adams & Guava": 0.2, "Jefferson & Fig": 0.5},
    "Jefferson & Fig": {"Jefferson & Guava": 0.5, "Jefferson & Elm": 0.5},
    "Jefferson & Elm": {"Jefferson & Fig": 0.5, "Jefferson & Pear/Eastman": 0.4},
    "Jefferson & Pear/Eastman": {"Jefferson & Elm": 0.4, "Jefferson & Cherry": 0.4},
    "Jefferson & Cherry": {"Jefferson & Pear/Eastman": 0.4},
    "Sierra Vista Shopping Center": {"Hancock & Calle Yorba Vista": 0.2, "Hancock & Las Brisas S/Coliseum": 0.2, "Los Alamos & Parkcrest": 0.3, "Murrieta Gateway": 0.4},
    "Hancock & Las Brisas N/Ridgewood": {"Hancock & Leafwood/Milkwood": 0.5, "California Oaks & Hancock": 0.3},
    "Hancock & Leafwood/Milkwood": {"Hancock & Las Brisas S/Coliseum": 0.3, "Hancock & Las Brisas N/Ridgewood": 0.5},
    "Hancock & Las Brisas S/Coliseum": {"Hancock & Calle Yorba Vista": 0.2, "Sierra Vista Shopping Center": 0.2, "Hancock & Leafwood/Milkwood": 0.3, "Los Alamos & Parkcrest": 0.3},
    "Los Alamos & Parkcrest": {"Los Alamos & Lincoln/Vista Murrieta": 0.3, "Hancock & Las Brisas S/Coliseum": 0.3, "Sierra Vista Shopping Center": 0.3, "Hancock & Calle Yorba Vista": 0.3, "Hancock & Parkcrest": 0.3},
    "Los Alamos & Lincoln/Vista Murrieta": {"Los Alamos & Parkcrest": 0.3, "Lincoln & Wichita": 0.5, "Los Alamos & Monroe": 0.4},
    "Los Alamos & Monroe": {"Los Alamos & Lincoln/Vista Murrieta": 0.4, "Murrieta Mesa High School": 0.2, "Los Alamos & Madison": 0.5},
    "Los Alamos & Madison": {"Madison @ Mulligans": 0.1, "Madison Square & Murrieta Spectrum Shopping Center": 0.1, "Jefferson & Ivy/Los Alamos": 0.3, "Los Alamos & Monroe": 0.5},
    "Madison Center": {"Madison & Juniper": 0.1, "Kalmia @ Kohl's": 0.3, "California Oaks Plaza": 0.5},
    "Madison & Juniper": {"Madison Center": 0.1, "Madison @ Mulligans": 0.2, "Jefferson & Juniper": 0.2},
    "Madison @ Mulligans": {"Madison & Juniper": 0.2, "Los Alamos & Madison": 0.1},
    "Monroe & Sunflower": {"Monroe & Symphony Park": 0.2, "California Oaks Plaza": 0.2, "California Oaks & Jackson": 0.3, "Calvary Chapel Murrieta": 0.6},
    "Monroe & Symphony Park": {"Monroe & Sunflower": 0.2, "Murrieta Mesa High School": 0.3},
    "Murrieta Mesa High School": {"Monroe & Symphony Park": 0.3, "Los Alamos & Monroe": 0.2},
    "Madison Square & Murrieta Spectrum Shopping Center": {"Los Alamos & Madison": 0.1, "Murrieta Springs Plaza": 0.2},
    "Murrieta Springs Plaza": {"Madison Square & Murrieta Spectrum Shopping Center": 0.2, "Jefferson & Murrieta Hot Springs": 0.4, "Rancho Springs Medical Center": 1.0, "Murrieta Hot Springs & Alta Murrieta Sam's Club": 1.4},
    "Hancock & Calle Yorba Vista": {"Sierra Vista Shopping Center": 0.2, "Hancock & Las Brisas S/Coliseum": 0.2, "Los Alamos & Parkcrest": 0.3, "Hancock & Parkcrest": 0.3},
    "Hancock & Parkcrest": {"Hancock & Calle Yorba Vista": 0.3, "Los Alamos & Parkcrest": 0.3, "Rady Children's Specialists": 0.1},
    "Rady Children's Specialists": {"Hancock & Parkcrest": 0.1, "Rancho Springs Medical Center": 0.5},
    "Rancho Springs Medical Center": {"Murrieta Springs Plaza": 1.0, "Rady Children's Specialists": 0.5, "Murrieta Hot Springs & Alta Murrieta Sam's Club": 0.5},
    "Buchanan Elementary School": {"Torrey Pines & Sugarberry": 0.4, "Margarita & Torrey Pines": 0.3},
    "Torrey Pines & Sugarberry": {"Buchanan Elementary School": 0.4},
    "Lincoln & Wichita": {"Los Alamos & Lincoln/Vista Murrieta": 0.5, "Shivela Middle School": 0.3},
    "Shivela Middle School": {"Lincoln & Wichita": 0.3, "California Oaks Sports Park": 0.2}
    }

## Dijkstra's Algorithm

In [19]:
# This interpretation of Dijkstra's Algorithm is originally from: https://algodaily.com/lessons/an-illustrated-guide-to-dijkstras-algorithm/python 

# Source and destination to determine the route
# View the dataset to input your own values
source = "Whitewood Park"
destination = "Antelope Hills Elementary School"

def dijkstra():
  # Declaring variables and lists
  unvisited = graph
  shortest_distances = {}
  route = []
  path_nodes = {}

  # For loop to iterate through the unvisited nodes while searching for best path
  for nodes in unvisited:
      shortest_distances[nodes] = math.inf
  shortest_distances[source] = 0

  # While the node is unvisited, each should be checked and the node with the shortest distance should be chosen
  while unvisited:
      min_node = None
      for current_node in unvisited:
          if min_node is None:
              min_node = current_node
          elif shortest_distances[min_node] \
              > shortest_distances[current_node]:
              min_node = current_node
      for (node, value) in unvisited[min_node].items():
          if value + shortest_distances[min_node] \
              < shortest_distances[node]:
              shortest_distances[node] = value \
                  + shortest_distances[min_node]
              path_nodes[node] = min_node
      unvisited.pop(min_node)
  node = destination

  # Continue searching unless destination is reached or if the path is unreachable
  while node != source:
      try:
          route.insert(0, node)
          node = path_nodes[node]
      except Exception:
          print('Path not reachable')
          break
  route.insert(0, source)

  # Display the path with the shortest distance calculation
  if shortest_distances[destination] != math.inf:
      print('The shortest distance is ' + str(shortest_distances[destination]) + ' miles.')
      print('The path is :' + str(route))

dijkstra()

The shortest distance is 4.5 miles.
The path is :['Whitewood Park', 'Whitewood & Blackthorne', 'Whitewood & Willowbend', 'Rail Ranch Elementary School', 'Los Alamos Hills Sports Park', 'Whitewood & Hunter', 'Clinton Keith & Whitewood', 'Clinton Keith @ Costco & Vista Murrieta High School', 'Orchard Center', 'Antelope Hills Elementary School']


# Map

---

Folium & Geoapify

## Import Libraries

In [20]:
### GEOAPIFY
from geopy.geocoders import Nominatim
from shapely.geometry import Point, LineString
import folium 
import matplotlib.pyplot as plt
import random
import requests
import itertools
from requests.structures import CaseInsensitiveDict

# API token
url = "https://api.geoapify.com/v1/routing?waypoints=50.96209827745463%2C4.414458883409225%7C50.429137079078345%2C5.00088081232559&mode=drive&apiKey=f3d3263adaa04715addde96fd3f64e7b"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

response = requests.get(url, headers=headers)

# Checking status of Geoapify API token
print(response.status_code)

200


## Folium

In [25]:
# Defining API key variable for use within the API request URL
api_key = "f3d3263adaa04715addde96fd3f64e7b"

routes = {
    "Route 1" : [ "Heritage Marketplace", "Keller & Whitewood", "Poinsettia & Verbena", "Mapleton & Iris", "Mapleton Park", "Meadowlark & Brians", "Winchester & Max Gilliss/Thompson", "Murrieta Hot Springs & Winchester (Hwy 79)" ],
    "Route 2" : [ "Clinton Keith & Palomar", "Washington & Calle Del Oso Oro/Nutmeg", "Glen Arbor Park", "California Oaks & Hancock", "Rancho Springs Medical Center", "Margarita & Torrey Pines" ],
    "Route 3" : [ "Kalmia @ Kohl's", "Murrieta Hot Springs & Winchester (Hwy 79)", "Winchester & Hunter", "Winchester & La Alba/Sparkman", "French Valley Airport @ Winchester", "French Valley Village Shopping Center" ],
    "Route 4" : [ "Whitewood & Lee/Linnel", "Whitewood & Mapleton", "Clinton Keith @ Costco & Vista Murrieta High School", "Washington & Calle Del Oso Oro/Nutmeg", "Washington & Hawthorn" ],
    "Route 5" : [ "Kalmia & Hayes", "Clinton Keith & California Oaks", "Clinton Keith & Menifee", "Los Alamos & Shannon N", "Hayes & Ivy", "Kalmia & Hayes" ],
    "Route 6" : [ "Murrieta Palms @ Ynez", "Firefighters Park", "Alta Murrieta Elementary School", "Alta Murrieta Sports Park", "Murrieta Hot Springs & Alta Murrieta (Sam's Club)", "Firefighters Park", "McElwain @ Super Target" ],
    "Route 7" : [ "Murrieta Town Square", "Clinton Keith & California Oaks", "Monte Vista Elementary School", "Hunter & Via Mira Mosa", "Margarita & Torrey Pines" ],
    "Route 8" : [ "Jefferson & Cherry", "Jefferson & Guava", "Hayes & De Luz/Guava", "Kalmia & Hayes", "Clinton Keith & California Oaks" ],
    "Route 9" : [ "Renaissance Plaza", "Inland Valley Medical Center", "Glen Arbor Park", "California Oaks Sports Park", "Los Alamos & Lincoln/Vista Murrieta" ],
    "Route 10" : [ "Riverside County Fire Department Station 75", "Jefferson & Cherry" ],
    "Route 11" : [ "Heritage Marketplace", "Jefferson & Cherry" ],
    "Route 12" : [ "Riverside County Fire Department Station 75", "Winchester & Max Gilliss/Thompson" ]
}

def get_bus_stop_info(name):
  index = data[data["Stop Name"] == name].index
  lat = float(data.iloc[index]["Latitude"])
  lon = float(data.iloc[index]["Longitude"])
  cla = str(data.iloc[index]["Classification"])
  return (lat, lon, cla)

# Function to create the map
def create_map(routes, colors, show_stops=True):
  
  # Initialize map at the location of Murrieta, California, zoomed to display all city limits
  map = folium.Map(location=[33.583754, -117.186731], zoom_start=13)
  headers = CaseInsensitiveDict()
  headers["Accept"] = "application/json"

  # Converting the bus route name to latitude and longitude
  for route_name, bus_stops in routes.items():
    geoapify_points = []
    url = ""

    # For loop to format the API request URL
    for bus_stop in bus_stops:
      (lat, lon, cla) = get_bus_stop_info(bus_stop)
      url += str(lat)
      url += ","
      url += str(lon)
      url += "|" # Remove this character for last point to fit the API request URL
    geoapify_points.append(url[:-1])
    geoapify_points = ''.join(geoapify_points)

    # API token containing source and destination coordinates for route plotting
    complete_url = f"https://api.geoapify.com/v1/routing?waypoints={geoapify_points}&mode=drive&avoid=highways&apiKey={api_key}"

    # Calling API request
    response = requests.get(complete_url, headers=headers)
    inbetween_points = response.json()['features'][0]['geometry']['coordinates']

    points = []
    for point in inbetween_points:
      for p in point:
        points.append((p[1], p[0]))

    # Add the lines to the map
    # Change opacity variable to alter visability of overlapping routes
    folium.PolyLine(points, weight=5, opacity=0.75, color=next(colors)).add_to(map)

    # Draw map markers
    if show_stops:
      # Create bus stop markers and add them for each specifically defined nodes
      for bus_stop in bus_stops:
        (lat, lon, cla) = get_bus_stop_info(bus_stop)
        folium.Marker(location=[lat,lon], popup=bus_stop).add_to(map)
    else:
      # Add marker for bus route in the middle, calculating the midpoint of each route
      middle_stop = math.floor(len(points) / 2) 
      middle_stop_pos = points[middle_stop]
      # Plotting a marker on the midpoint
      folium.Marker(location=[middle_stop_pos[0], middle_stop_pos[1]], popup=route_name).add_to(map)

  return map

# Color cycle for different route colors
colors = itertools.cycle(['navy', 'darkturquoise', 'rebeccapurple',' mediumaquamarine', 'darkgreen', 'palegreen', 'yellowgreen', 'gold', 'peru', 'tomato', 'lightcoral', 'maroon'])

### Map settings below
# show_stops = True to display the nodes within the path search
# show_stops = False to display the midpoint of each route
map = create_map(routes, colors, show_stops=False)
map

In [ ]:
### Uncomment code block to view all bus stops
# Zip the values of the name of the bus stop and its corresponding coordinates in order to display all of the bus stops at once

# for name, lat, lon in zip(name, lat, lon):
#   folium.Marker(location=[lat,lon], popup=name).add_to(map)
# map